In [1]:
import os
import sys
p = os.path.abspath('../..')
if p not in sys.path:
    sys.path.append(p)

from waveorder.io.writer import WaveorderWriter
import numpy as np
import zarr

### Initialize the writer and set the position

In [2]:
writer = WaveorderWriter('/Users/cameron.foltz/Desktop/zarr_test', 'physical')

In [3]:
writer.set_position(2)
writer.create_zarr()

Found existing store at /Users/cameron.foltz/Desktop/zarr_test/Pos_002/physical_data.zarr


### Create some fake data

In [7]:
T = 3
C = 2
Z = 65
Y = 256
X = 256

data = np.random.rand(T, C, Z, Y, X)
# data = np.ones((T, C, Z, Y, X))

### initialize the array which sets the data saving parameters

In [9]:
writer.init_array(data_shape=data.shape, chunk_size=(1,1,1,256,256), dtype='float64')

### set the channel zarr_attributes

In [10]:
writer.set_channel_attributes(['DAPI', 'Phase3D'])

In [11]:
# print zarr_attributes dictionary
writer.store.attrs.asdict()

{'multiscales': [{'datasets': [{'path': 'array'}], 'version': '0.1'}],
 'omero': {'channels': [{'active': True,
    'coefficient': 1.0,
    'color': '808080',
    'family': 'linear',
    'inverted': False,
    'label': 'DAPI',
    'window': {'end': 65535.0, 'max': 65535.0, 'min': 0.0, 'start': 0.0}},
   {'active': True,
    'coefficient': 1.0,
    'color': '808080',
    'family': 'linear',
    'inverted': False,
    'label': 'Phase3D',
    'window': {'end': 65535.0, 'max': 65535.0, 'min': 0.0, 'start': 0.0}}]},
 'rdefs': {'defaultT': 0,
  'model': 'color',
  'projection': 'normal',
  'defaultZ': 0}}

### Write the data along specific dimensions

In [13]:
writer.write(data, T=[0,3], C=[0,2], Z=[0,65])